In [67]:
import spacy
import pickle
import random
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from spacy.training.example import Example
!pip install PyMuPDF
import sys, fitz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 33.2 MB/s eta 0:00:00


In [56]:
print(spacy.__version__)

3.6.1


In [57]:
train_data = pickle.load(open('/content/train_data.pkl', 'rb'))

In [58]:
nlp = spacy.blank('en')

In [59]:
def train_model(nlp, train_data):
    # Check if the NER component is already in the pipeline
    if 'ner' not in nlp.pipe_names:
        ner = nlp.add_pipe('ner', last=True)
    else:
        ner = nlp.get_pipe('ner')

    # Add labels to the NER component
    for _, annotations in train_data:
        for ent in annotations['entities']:
            ner.add_label(ent[2])

    # Disable other pipelines
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()

        for epoch in range(10):
            print(f"Iteration {epoch} starting...")
            random.shuffle(train_data)
            losses = {}

            for text, annotations in train_data:
                try:
                    example = Example.from_dict(nlp.make_doc(text), annotations)
                    nlp.update([example], drop=0.5, losses=losses, sgd=optimizer)
                except Exception as e:
                    pass

            print(losses)

# Train the model
train_model(nlp, train_data)

Iteration 0 starting...
{'ner': 9754.011212417252}
Iteration 1 starting...
{'ner': 2526.3056880638883}
Iteration 2 starting...
{'ner': 3008.422235473373}
Iteration 3 starting...
{'ner': 2567.606096365564}
Iteration 4 starting...
{'ner': 3618.537945499982}
Iteration 5 starting...
{'ner': 3008.6558368077945}
Iteration 6 starting...
{'ner': 2187.97859475291}
Iteration 7 starting...
{'ner': 2222.87023687919}
Iteration 8 starting...
{'ner': 2781.973001506483}
Iteration 9 starting...
{'ner': 2199.890924898274}


In [60]:
nlp.to_disk('nlp_model2')

In [61]:
nlp_model = spacy.load('/content/nlp_model2')

In [62]:
train_data[0][0]

'Navas Koya Test Engineer  Mangalore, Karnataka - Email me on Indeed: indeed.com/r/Navas-Koya/23c1e4e94779b465  Willing to relocate to: Mangalore, Karnataka - Bangalore, Karnataka - Chennai, Tamil Nadu  WORK EXPERIENCE  System Engineer  Infosys -  August 2014 to Present  .NET application Maintenance and do the code changes if required  Test Engineer  Infosys -  June 2015 to February 2016  PrProject 2:  Title: RBS W&G Proving testing. Technology: Manual testing Role: Software Test Engineer  Domain: Banking Description:  Write test cases & descriptions. Review the entries. Upload and map the documents into HP QC. Execute the testing operations in TPROD mainframe. Upload the result in QC along with the proof. Roles and Responsibilities: •Prepared the Test Scenarios  •Prepared and Executed Test Cases •Performed functional, Regression testing, Sanity testing.  •Reviewed the Test Reports and Preparing Test Summary Report. •Upload Test cases to the QC. •Execute in TPROD Mainframe. •Defect Tra

In [63]:
doc = nlp_model(train_data[0][0])
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Navas Koya
LOCATION                      - Mangalore
LOCATION                      - Mangalore
LOCATION                      - Chennai
DESIGNATION                   - System Engineer
COMPANIES WORKED AT           - Infosys
COMPANIES WORKED AT           - Infosys
COMPANIES WORKED AT           - Infosys Limited
DEGREE                        - Bachelor of Computer Applications
LOCATION                      - Mangalore
DEGREE                        - Bachelor of computer application:
LOCATION                      - Mangalore


In [69]:
fname = '/content/Smith Resume.pdf'
doc = fitz.open(fname)
text = ""
for page in doc:
    text += page.get_text("text")
#replace new line with space
tx = " ".join(text.split('\n'))
print(tx)

Michael Smith  BI / Big Data/ Azure  Manchester, UK- Email me on Indeed: indeed.com/r/falicent/140749dace5dc26f    10+ years of Experience in Designing, Development, Administration, Analysis,  Management  inthe  Business  Intelligence  Data  warehousing,  Client  Server  Technologies, Web-based Applications, cloud solutions and Databases.  Data warehouse: Data analysis, star/ snow flake schema data modeling and design  specific todata warehousing and business intelligence environment.  Database: Experience in database designing, scalability, back-up and recovery,  writing andoptimizing SQL code and Stored Procedures, creating functions, views,  triggers and indexes.   Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL  Azure, StreamAnalytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure  data lake analytics(U-SQL).  Big Data: Worked Azure data lake store/analytics for big data processing and Azure  data factoryto schedule U-SQL jobs. Designed and d

In [70]:
doc = nlp_model(tx)
#Formatting label and text
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Michael Smith
LOCATION                      - Manchester
COMPANIES WORKED AT           - Microsoft
LOCATION                      - Manchester
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COLLEGE NAME                  - The University of Manchester - UK  2007    SKILLS  problem solving (Less than 1 year), project lifecycle (Less than 1 year), project  manager (Less than 1 year), technical assistance. (Less than 1 year)  ADDITIONAL INFORMATION  Professional Skills  Excellent analytical, problem solving, communication, knowledge transfer and  interpersonalskills with ability to interact with individuals at all the levels  Quick learner and maintains cordial relationship with project manager and team  members andgood performer both in team and independent